In [9]:
import boto3
import json
from botocore.exceptions import ClientError

In [10]:
session = boto3.Session(profile_name='workshop')
config = session.client('config', region_name='us-east-1')

In [11]:
def create_custom_rule(rule_name, rule_description, resource_type, rule_content, input_parameters=None):
    rule = {
        'ConfigRuleName': rule_name,
        'Description': rule_description,
        'Scope': {
            'ComplianceResourceTypes': [
                resource_type,
            ],
        },
        'Source': {
            'Owner': 'CUSTOM_POLICY',
            'SourceDetails': [
                {
                    'EventSource': 'aws.config',
                    'MessageType': 'ConfigurationItemChangeNotification'
                },
            ],
            'CustomPolicyDetails': {
                'PolicyRuntime': 'guard-2.x.x',
                'PolicyText': rule_content,
                'EnableDebugLogDelivery': True,
            },
        },
        'InputParameters' :input_parameters
    }
    try:
        response = config.put_config_rule(ConfigRule=rule)
        print('Successfully created Config rule: ' + json.dumps(response, indent=2, default=str))
    except ClientError as e:
        print('Error creating Config rule: ' + e.response['Error']['Code'])
    except Exception as e:
        print('Error creating Config rule: ' + str(e))

In [4]:
rule_name = 's3-bucket-versioning-enabled-test'
rule_description = 'Checks that S3 buckets have versioning enabled'
resource_type = 'AWS::S3::Bucket'
rule_content = """rule check_bucketversioning {
     supplementaryConfiguration.BucketVersioningConfiguration.status == "Enabled" <<
     result: NON_COMPLIANT
     message: S3 Bucket Versioning is NOT enabled.
     >>
}
"""

In [12]:
rule_name = 'ec2-eip-running-test-new'
rule_description = 'Checks that EC2 EIPs are associated with running instances'
resource_type = 'AWS::EC2::Instance'
input_parameters = "{\"volumeType\":\"gp3\"}"
rule_content = """let eipresource = relationships.*[ resourceType  == 'AWS::EC2::EIP' ]


rule check_ec2_eip_compliance {
    when %eipresource !empty {
    configuration.state.name == "running"
}
}
"""
create_custom_rule(rule_name, rule_description, resource_type, rule_content, input_parameters)

Successfully created Config rule: {
  "ResponseMetadata": {
    "RequestId": "7bf5d8e3-ec68-4f91-b48d-4187d809f3a1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "7bf5d8e3-ec68-4f91-b48d-4187d809f3a1",
      "strict-transport-security": "max-age=86400",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "0",
      "date": "Wed, 31 May 2023 01:28:54 GMT"
    },
    "RetryAttempts": 0
  }
}
